In [ ]:
from modelos.modelos import comunio_pred_lib
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from lightgbm import LGBMRegressor as LGBR
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense
pd.set_option('display.max_columns',None)
pd.set_option('display.max_rows',None)

## Primero obtenemos los datos para entrenar los modelos

    Para ello utilizo las funciones en el archivo comunio_pred_lib.
    - create_data_train genera un data frame por cada jornada que reciba como argumento
    - preprocess_data recibe el data frame resultante de la operación de antes y nos genera
      las variables necesarias para entrenar nuestros modelos.
          - X: Datos numéricos de nuestro data frame inicial sin el target
          - y: Target que recibirá nuestro modelo
          - X_train, X_test, y_train, y_test : Datos resultantes de realizar la operación train_test_split
          - X_train_s, X_test_s, y_train_s, y_test_s: Datos anteriores normalizados
          - x_scaler, y_scaler: Variable necesaria para poder transformar los datos en el futuro

In [ ]:
# List of journeys to create a full data frame to train de models

j = [19,20,21,23,25,26,27,28,29,30,31,32]

df = pd.DataFrame()
for journey in j:
    print('Añadiendo Jornada',journey)
    if journey != 21 and journey != 23:
        
        df_2 = comunio_pred_lib.create_data_train(journey)

        n_j = journey+1
        
        next_j = comunio_pred_lib.create_data_train(n_j)
        next_j = next_j[['Player','J_Actual']]
        next_j_target= next_j.rename(columns={'Player': 'Jugador', 'J_Actual':'Target'})
        df_3 = df_2.merge(next_j_target, how='left', left_on='Player', right_on='Jugador')
        df_3 = df_3.dropna()
            
        
    
    else:
        
        df_2 = comunio_pred_lib.create_data_train(journey)
        n_j = journey+2
        
        next_j = comunio_pred_lib.create_data_train(n_j)
        next_j = next_j[['Player','J_1']]
        next_j_target= next_j.rename(columns={'Player': 'Jugador', 'J_1':'Target'})
        df_3 = df_2.merge(next_j_target, how='left', left_on='Player', right_on='Jugador')
        df_3 = df_3.dropna()

    df = df.append(df_3)


In [ ]:
X, y, X_train, X_test, y_train, y_test ,X_train_s, X_test_s, y_train_s, y_test_s, x_scaler, y_scaler = comunio_pred_lib.preprocess_data(df)

In [ ]:
rfr = RandomForestRegressor(max_depth= 5, max_leaf_nodes= 35, n_estimators= 300, n_jobs= -1)
xgbr = XGBRegressor(max_depth=5, n_estimators=500,learning_rate=0.01, n_jobs=-1, gamma=0.1 )
gb = GradientBoostingRegressor(max_depth=5, n_estimators=400,learning_rate=0.001, alpha=0.8, max_leaf_nodes=35)
models = (rfr,xgbr,gb)
for model in models:
    name = str(model)[:13]
  
    model.fit(X_train_s,y_train)
    model.score(X_train_s, y_train)
    pred = model.predict(X_test_s)
    mse = mean_squared_error(pred,y_test, squared=False)
    
    
    print (f'mse del modelo {name} = {mse}')

### Reentrenamos los modelos ahora con todos los datos para predecir jornadas venideras

In [ ]:
rfr.fit(X,y)
xgbr.fit(X,y)
gb.fit(X,y)
'Modelos entrenados'

### Se entrenaron dos modelos de redes neuronales con los mismos parámetros salvo las épocas.
    Ejemplo de uno de ellos.
    Se entreno con 10 y 20 epocas
    rnn con 20 epocas y rnn2 con 10 epocas

In [ ]:
rnn =  Sequential()

rnn.add(Dense(500, activation='relu',input_dim=27))
rnn.add(Dense(250, activation='relu'))
rnn.add(Dense(125, activation='relu'))
rnn.add(Dense(60, activation='relu'))
rnn.add(Dense(1, activation='linear'))

rnn.compile(loss="mean_squared_error",
             optimizer="adam")
his = rnn.fit(X_train_s,
        y_train_s,
        epochs=20,
        validation_split=0.2
        )
pred_rnn = rnn.predict(X_test_s)
pred2_rnn = y_scaler.inverse_transform(pred_rnn)
mse = mean_squared_error(pred2_rnn,y_test, squared=False)

        
pd.DataFrame(his.history).plot(figsize=(8,5))
plt.grid()
plt.show()

In [ ]:
print (f'mse del modelo rnn = {mse}')

## Con los modelos ya entrenados realizamos las predicciones

    Vamos utilizar los datos de la jornada 33 para predecir la jornada 34

In [ ]:
j_33 = comunio_pred_lib.create_data_train(33)
j_34 = comunio_pred_lib.create_data_train(34)
pred_data = j_33._get_numeric_data()

In [ ]:
j_33.info()

### Realizamos las predicciones usando las funciones de nuestra librería

    Todas las funciones son similares, reciben un dataframe y devuelven las prediciones de ese df.

In [ ]:
pred_rfr = comunio_pred_lib.predict_rf(pred_data)
pred_xgb = comunio_pred_lib.predict_xgb(pred_data)
pred_gb = comunio_pred_lib.predict_gb(pred_data)
pred_rnn = comunio_pred_lib.predict_rnn(pred_data)
pred_rnn2 = comunio_pred_lib.predict_rnn2(pred_data)

### Creamos un dataframe con las predicciones de nuestros modelos y añadimos la realidad para comparar

In [ ]:
preds = pd.DataFrame(j_33['Player'])
preds['Position'] = j_33.Position
preds['rfr'] = pred_rfr
preds['xgb'] = pred_xgb
preds['gb'] = pred_gb
preds['rnn'] = pred_rnn
preds['rnn2'] = pred_rnn2
preds = preds.merge(j_34[['Player','J_Actual']], left_on='Player', right_on='Player')
preds = round(preds)
preds['avg_l_5_J'] = j_33.Avg_last_5_games

preds

### Ahora vamos a explorar un poco los resultados y obtener los 11 ideales para la J 34

    Para utilizamos las funciones de once_ideal_(modelo) para obtener el once ideal previsto por ese modelo.
    Disponemos de 5 funciones:            
    
    Cada función recibe unos datos y la disposición de la alineación, numero de defensas, medios atacantes.
    
        - once_ideal_rf (Usa el modelo random forest)
        - once_ideal_xgb (Usa el modelo xgboot)
        - once_ideal_gb (Usa el modelo gradient boost)
        - once_ideal_rnn (Usa el modelo red neuronal con 20 épocas)
        - once_ideal_rnn2 (Usa el modelo red neuronal con 10 épocas)

### Distribución de los puntos de todos los jugadores en la J 34

In [ ]:
plt.hist(j_34['J_Actual'], bins=10);
plt.title('Distribución puntos Jornada 34')
plt.xlabel('Ptos el jugador');

#### Distribución de las predicciones de cada uno de los modelos

In [ ]:
plt.hist(round(preds.rfr), bins=8);
plt.title('Distribución predicciones Random Forest')
plt.xlabel('Predicción ptos el jugador');

In [ ]:
plt.hist(round(preds.xgb), bins=8);
plt.title('Distribución predicciones XGBoost')
plt.xlabel('Predicción ptos el jugador');

In [ ]:
plt.hist(round(preds.gb), bins=8);
plt.title('Distribución predicciones Gradient Boost')
plt.xlabel('Predicción ptos el jugador');

In [ ]:
plt.hist(round(preds.rnn), bins=8);
plt.title('Distribución predicciones Red Neuronal')
plt.xlabel('Predicción ptos el jugador');


In [ ]:
plt.hist(round(preds.rnn2), bins=8);
plt.title('Distribución predicciones Red Neuronal 2')
plt.xlabel('Predicción ptos el jugador');

### Generamos los onces ideales de cada equipo y comparamos la predicción con los datos reales

In [ ]:
eleven_rf = comunio_pred_lib.once_ideal_rf(preds,3,4,3)
eleven_xgb = comunio_pred_lib.once_ideal_xbgr(preds,3,4,3)
eleven_gb = comunio_pred_lib.once_ideal_gb(preds,3,4,3)
eleven_rnn = comunio_pred_lib.once_ideal_rnn(preds,3,4,3)
eleven_rnn2 = comunio_pred_lib.once_ideal_rnn2(preds,3,4,3)

In [ ]:
print('Totales xgb: ', sum(eleven_xgb.xgb), ', real: ', 
      sum(eleven_xgb.J_Actual)
     )
print('Totales rfr: ', sum(eleven_rf.rfr), ', real: ', 
      sum(eleven_rf.J_Actual)
     )
print('Totales gb: ', sum(eleven_gb.gb), ', real: ', 
      sum(eleven_gb.J_Actual)
     )
print('Totales rnn: ', sum(eleven_rnn.rnn), ', real: ', 
      sum(eleven_rnn.J_Actual)
     )
print('Totales rnn2: ', sum(eleven_rnn2.rnn2), ', real: ', 
      sum(eleven_rnn2.J_Actual)
     )

In [ ]:
eleven_gb

In [ ]:
eleven_rnn

### Ahora vamos a testear con una plantilla real la mía del comunio a ver que alineación recomienda

    Primero generamos un df a partir de los datos de prediciones que tenemos, y los filtramos con los jugadores que tengo en mi plantilla de comunio

In [ ]:
squad = ['Courtois', 'Bono',
         'Balliu', 
         'Fran García',
         'Maffeo',
         'Ronald Araujo', 
         'Alderete', 'Manu Sánchez', 
         #'Giménez', Está lesionado o tenía pocas posibilidades de jugar
         #'Yéremi',  
         'Gavi',  
         #'Modric',  
         'Brais', 'Papu Gómez', 'Dani Rodríguez', 'Pere Milla', 'Cervi', 'Berenguer',
         'Vinícius Júnior', 'Aubameyang', 
         #'Sancet',  
         'Morales']
df_squad_kike = pd.DataFrame()
for player in squad:
    #print (player)
    for p in preds.Player:
        if player in p:
            #print('encontrado')
            add_player = pd.DataFrame(preds.loc[preds.Player==p])
            #print(add_player)
            df_squad_kike = df_squad_kike.append(add_player)
print(len(squad),len(df_squad_kike))

df_squad_kike.at[520,'Position'] = 'MD'
df_squad_kike = round(df_squad_kike)
df_squad_kike

In [ ]:
xgb_eleven_kike = round(comunio_pred_lib.once_ideal_xbgr(df_squad_kike,3,5,2))

rf_eleven_kike = round(comunio_pred_lib.once_ideal_rf(df_squad_kike,3,5,2))

gb_eleven_kike = round(comunio_pred_lib.once_ideal_gb(df_squad_kike,3,5,2))

rnn_eleven_kike = round(comunio_pred_lib.once_ideal_rnn(df_squad_kike,3,5,2))

rnn2_eleven_kike = round(comunio_pred_lib.once_ideal_rnn2(df_squad_kike,3,5,2))

print('Totales xgb: ', sum(xgb_eleven_kike.xgb), ', real: ', 
      sum(xgb_eleven_kike.J_Actual)
     )
print('Totales rfr: ', sum(rf_eleven_kike.rfr), ', real: ', 
      sum(rf_eleven_kike.J_Actual)
     )
print('Totales gb: ', sum(gb_eleven_kike.gb), ', real: ', 
      sum(gb_eleven_kike.J_Actual)
     )
print('Totales rnn: ', sum(rnn_eleven_kike.rnn), ', real: ', 
      sum(rnn_eleven_kike.J_Actual)
     )
print('Totales rnn2: ', sum(rnn2_eleven_kike.rnn2), ', real: ', 
      sum(rnn2_eleven_kike.J_Actual)
     )

In [ ]:
gb_eleven_kike


In [ ]:
rnn2_eleven_kike

## Conclusiones

El peor modelo es sin duda el gradient boost, ya que prácticamente asigna la misma puntución a todos los jugadores.
El resto de modelos funcionan bastante bien y aunque el rnn2 no es el que mejores resultados obtiene en cuanto error absoluto es el que mejor generaliza y se acerca más a la realidad, solo hay que comparar las distribuciones de la puntuación real y de la rnn2 para ver que son las más similares.

In [ ]:
sum(df_squad_kike.rnn2)